In [23]:
#import the essential packages
#base packages
import numpy as np
import pandas as pd
import geopandas as gpd

# Plotting packages
import seaborn as sns
from matplotlib import pyplot as plt
import holoviews as hv
import hvplot.pandas

# Sodapy API packages
import requests
from sodapy import Socrata

# Set a Random Seed
np.random.seed(42)
pd.options.display.max_columns = 999

In [8]:
# Models
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# Model selection
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

# Pipelines
from sklearn.pipeline import make_pipeline

# Preprocessing
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [22]:
distracts_raw = pd.read_csv("C://Users//zhaiy//Downloads//Planning_Districts.csv")

distracts_raw.head()

,OBJECTID_1,OBJECTID,DIST_NAME,ABBREV,Shape__Area,Shape__Length,PlanningDist,DaytimePop
0,1,14,River Wards,RW,2.107270e+08,66931.595020,NaN,NaN
1,2,3,North Delaware,NDEL,2.700915e+08,89213.074378,NaN,NaN
2,3,0,Lower Far Northeast,LFNE,3.068529e+08,92703.285159,NaN,NaN
3,4,9,Central,CTR,1.782880e+08,71405.143450,NaN,NaN
4,5,10,University Southwest,USW,1.296468e+08,65267.676141,NaN,NaN


In [4]:
# the CARTO API url
carto_url = "https://phl.carto.com/api/v2/sql"

# Only pull 2022 sales for single family residential properties
where = "sale_date >= '2022-01-01' and sale_date <= '2022-12-31'"
where = where + " and category_code_description IN ('SINGLE FAMILY', 'Single Family')"

# Create the query
query = f"SELECT * FROM opa_properties_public WHERE {where}"

# Make the request
params = {"q": query, "format": "geojson", "where": where}
response = requests.get(carto_url, params=params)

# Make the GeoDataFrame
salesRaw = gpd.GeoDataFrame.from_features(response.json(), crs="EPSG:4326")

salesRaw.head()

,geometry,cartodb_id,assessment_date,basements,beginning_point,book_and_page,building_code,building_code_description,category_code,category_code_description,census_tract,central_air,cross_reference,date_exterior_condition,depth,exempt_building,exempt_land,exterior_condition,fireplaces,frontage,fuel,garage_spaces,garage_type,general_construction,geographic_ward,homestead_exemption,house_extension,house_number,interior_condition,location,mailing_address_1,mailing_address_2,mailing_care_of,mailing_city_state,mailing_street,mailing_zip,market_value,market_value_date,number_of_bathrooms,number_of_bedrooms,number_of_rooms,number_stories,off_street_open,other_building,owner_1,owner_2,parcel_number,parcel_shape,quality_grade,recording_date,registry_number,sale_date,sale_price,separate_utilities,sewer,site_type,state_code,street_code,street_designation,street_direction,street_name,suffix,taxable_building,taxable_land,topography,total_area,total_livable_area,type_heater,unfinished,unit,utility,view_type,year_built,year_built_estimate,zip_code,zoning,pin,building_code_new,building_code_description_new,objectid
0,POINT (-75.09742 40.04562),968,2022-05-24T00:00:00Z,None,180' N E BENNER ST,54278710,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,305,N,None,None,136.0,80000.0,0.0,4,0.0,16.0,None,0.0,None,A,35,80000,None,6123,4,6123 PALMETTO ST,None,None,None,PHILADELPHIA PA,6123 PALMETTO ST,19111-5729,159800.0,None,1.0,4.0,NaN,2.0,1577.0,None,KUNKEL DEXTER,None,352297800,E,C+,2024-03-13T00:00:00Z,114N100249,2022-05-11T00:00:00Z,1.0,None,None,None,PA,62860,ST,None,PALMETTO,None,47840.0,31960.0,F,2187.0,1360.0,H,None,None,None,I,1925,Y,19111,RSA5,1001410179,24,ROW PORCH FRONT,456845074
1,POINT (-75.18354 39.92504),1052,2022-05-24T00:00:00Z,D,114' W 22ND ST,54277717,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,37,None,None,None,50.0,0.0,0.0,4,0.0,14.0,None,0.0,None,A,48,0,None,2217,4,2217 JACKSON ST,SIMPLIFILE LC E-RECORDING,None,None,PHILADELPHIA PA,2217 JACKSON ST,19145-3316,153200.0,None,1.0,3.0,NaN,2.0,497.0,None,STEVANUS HARRY,None,482103500,E,C,2024-03-11T00:00:00Z,032S050163,2022-03-08T00:00:00Z,1.0,None,None,None,PA,45760,ST,None,JACKSON,None,122560.0,30640.0,F,700.0,1032.0,H,None,None,None,I,1920,Y,19145,RM1,1001291454,22,ROW TYPICAL,456845213
2,POINT (-75.23311 39.97710),1114,2023-05-09T00:00:00Z,None,102' N LANSDOWNE AVE,54278141,H30,SEMI/DET 2 STY MASONRY,1,SINGLE FAMILY,113,None,None,None,109.0,68480.0,11520.0,4,0.0,16.0,None,0.0,None,A,4,80000,None,1612,4,1612 N ALLISON ST,None,None,None,PHILADELPHIA PA,1612 N ALLISON ST,19131-3503,85600.0,None,1.0,4.0,NaN,2.0,2620.0,None,CHANEY WALTER JR,CHANEY REVA,041341700,E,C,2024-03-12T00:00:00Z,73N2 71,2022-08-08T00:00:00Z,1.0,None,None,None,PA,12200,ST,N,ALLISON,None,0.0,5600.0,F,1748.0,1527.0,None,None,None,None,I,1925,Y,19131,RM1,1001061205,32,TWIN CONVENTIONAL,456845162
3,POINT (-75.13930 40.00440),1730,2022-05-24T00:00:00Z,None,18' S TIOGA ST,54274443,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,199,None,None,None,70.0,0.0,0.0,4,0.0,14.0,None,0.0,None,A,43,0,None,3449,4,3449 N 6TH ST,SIERRA ABRAHAM,None,None,PHILADELPHIA PA,559 W ROOSEVELT BLV,19120,65500.0,None,1.0,3.0,NaN,2.0,1438.0,None,SIERRA ABRAHAM,None,431110100,E,C,2024-02-29T00:00:00Z,42N24 60,2022-11-20T00:00:00Z,15000.0,None,None,None,PA,87910,ST,N,6TH,None,52400.0,13100.0,F,980.0,1120.0,None,None,None,None,I,1920,Y,19140,RSA5,1001598446,22,ROW TYPICAL,456845772
4,POINT (-75.23190 39.93709),1868,2022-05-24T00:00:00Z,None,"321'7"" W OF 58TH ST",54266613,O30,ROW 2 STY MASONRY,1,SINGLE FAMILY,65,None,None,None,80.0,0.0,0.0,5,0.0,16.0,None,0.0,None,A,3,0,None,5840,4,5840 WARRINGTON AVE,None,None,None,PHILADELPHIA PA,5840 WARRINGTON AVE,19143-5216,96600.0,None,1.0,3.0,NaN,1.0,3551.0,None,WILLIAMS DARLENE,None,034084500,E,C,2024-02-02T00:00:00Z,028S050136,2022-04-24T00:00:00Z,1.0,None,None,None,PA,81500,AVE,None,WARRINGTON,None,77280.0,19320.0,F,1272.0,1208.0,None,None,None,None,I,1925,Y,19143,RSA5,1001552182,24,ROW PORCH FRONT,456846967


In [6]:
# The feature columns we want to use
cols = [
    "sale_price",
    "total_livable_area",
    "total_area",
    "garage_spaces",
    "fireplaces",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
    "exterior_condition",
    "zip_code",
]

# Trim to these columns and remove NaNs
sales = salesRaw[cols].dropna()

# Trim zip code to only the first five digits
sales['zip_code'] = sales['zip_code'].astype(str).str.slice(0, 5)

# Trim very low and very high sales
valid = (sales['sale_price'] > 3000) & (sales['sale_price'] < 1e6)
sales = sales.loc[valid]

sales.head()

,sale_price,total_livable_area,total_area,garage_spaces,fireplaces,number_of_bathrooms,number_of_bedrooms,number_stories,exterior_condition,zip_code
3,15000.0,1120.0,980.0,0.0,0.0,1.0,3.0,2.0,4,19140
14,45000.0,1024.0,843.0,0.0,0.0,1.0,3.0,2.0,4,19146
15,57500.0,1179.0,975.0,0.0,0.0,1.0,3.0,1.0,4,19151
16,30000.0,1638.0,1005.0,0.0,0.0,0.0,0.0,3.0,5,19139
19,83000.0,345.0,0.0,0.0,0.0,0.0,0.0,1.0,3,19107


In [9]:
# Split the data 70/30
train_set, test_set = train_test_split(sales, test_size=0.3, random_state=42)

# the target labels: log of sale price
y_train = np.log(train_set["sale_price"])
y_test = np.log(test_set["sale_price"])

# The features
feature_cols = [
    "total_livable_area",
    "total_area",
    "garage_spaces",
    "fireplaces",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
]
X_train = train_set[feature_cols].values
X_test = test_set[feature_cols].values

In [10]:
# Make a linear model pipeline
linear_pipeline = make_pipeline(StandardScaler(), LinearRegression())

# Fit on the training data
linear_pipeline.fit(X_train, y_train)

# What's the test score?
linear_pipeline.score(X_test, y_test)

0.24384984889930905

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [12]:
# Numerical columns
num_cols = [
    "total_livable_area",
    "total_area",
    "garage_spaces",
    "fireplaces",
    "number_of_bathrooms",
    "number_of_bedrooms",
    "number_stories",
]

# Categorical columns
cat_cols = ["exterior_condition", "zip_code"]

add a transformer

In [14]:
# Set up the column transformer with two transformers
# ----> Scale the numerical columns
# ----> One-hot encode the categorical columns

transformer = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ]
)

In [15]:
# Initialize the pipeline
# NOTE: only use 10 estimators here so it will run in a reasonable time
pipe = make_pipeline(
    transformer, RandomForestRegressor(n_estimators=10, 
                                       random_state=42)
)

In [16]:
# Fit the training set
pipe.fit(train_set, y_train);

# What's the test score?
pipe.score(test_set, y_test)

0.5336349258368888

In [17]:
# The column transformer...
transformer

ColumnTransformer(transformers=[('num', StandardScaler(),
                                 ['total_livable_area', 'total_area',
                                  'garage_spaces', 'fireplaces',
                                  'number_of_bathrooms', 'number_of_bedrooms',
                                  'number_stories']),
                                ('cat', OneHotEncoder(handle_unknown='ignore'),
                                 ['exterior_condition', 'zip_code'])])

In [29]:
# The steps in the column transformer
transformer.named_transformers_

{'num': StandardScaler(),
 'cat': OneHotEncoder(handle_unknown='ignore'),
 'remainder': 'drop'}

In [30]:
# The one-hot step
ohe = transformer.named_transformers_['cat']

# One column for each category type!
ohe_cols = ohe.get_feature_names_out()

# Full list of columns is numerical + one-hot 
features = num_cols + list(ohe_cols)